# 🗂️ MapReduce Tutorial
## 1. 🔍 What is MapReduce?
MapReduce is a programming model designed to process large-scale data in a distributed and parallel fashion.

It consists of three major phases:

- Map: Process input data and output intermediate key-value pairs.

- Shuffle/Sort: Group all intermediate values by their key.

- Reduce: Aggregate the values associated with each key to produce the final output​

## 2. MapReduce Use Case: Word Count
### 2.1 Map Phase
1. Read input data line by line.
2. Tokenize each line into words.
3. Emit (word, 1) for each word

In [1]:
def map_phase(input_file):
    """Reads lines from input_file, yields (word, 1) for each word."""
    with open(input_file, 'r') as f:
        for line in f:
            words = line.strip().split()
            for word in words:
                yield (word.lower(), 1)   # Convert to lower case

### 2.2 Shuffle/Sort Phase
We need to group by the key (the word). 

In [2]:
def shuffle_and_sort(mapped_data):
    """Groups the mapped data by key (word)."""
    sorted_data = {}
    for key, value in mapped_data:
        if key not in sorted_data:
            sorted_data[key] = []
        sorted_data[key].append(value)
    return sorted_data

### 2.3 Reduce Phase
For each word (key), we want to sum up the counts:

In [3]:
def reduce_phase(shuffled_data):
    """For each word key, sum up all the values."""
    reduced_data = {}
    for key, values in shuffled_data.items():
        reduced_data[key] = sum(values)
    return reduced_data

### 2.4 Putting It All Together

In [4]:
# 1. Map
mapped = map_phase("input.txt")

# 2. Shuffle/Sort
shuffled = shuffle_and_sort(mapped)

# 3. Reduce
reduced_result = reduce_phase(shuffled)

# Print or save results
for word, count in reduced_result.items():
    print(f"{word}: {count}")

mapreduce: 2
is: 3
a: 3
programming: 1
model: 1
for: 1
processing: 1
large: 1
data: 2
sets.: 1
it: 1
inspired: 1
by: 1
the: 4
map: 2
and: 2
reduce: 2
functions: 1
commonly: 1
used: 2
in: 2
functional: 1
programming.: 1
function: 2
applies: 1
transformation: 1
to: 1
each: 1
element: 1
of: 1
collection,: 1
while: 1
aggregates: 1
results.: 1
enables: 1
distributed: 1
computing: 1
across: 1
many: 1
machines.: 1
this: 1
approach: 1
widely: 1
big: 1
processing,: 1
including: 1
log: 1
analysis: 1
indexing.: 1


## 3. MapReduce Use Case: Calculate Mean and Variance
### 3.1 Map Phase: Each input data point emits `(value, value², count=1)`.

In [5]:
def map_phase(input_file):
    """Reads numbers from input_file and emits (x, x^2, 1) for each number."""
    with open(input_file, 'r') as f:
        for line in f:
            number = float(line.strip())  # Convert string to float
            yield (number, number ** 2, 1)  # Emit (x, x², count=1)

### 3.2 Shuffle/Sort: Groups all mapped values together.

In [6]:
def shuffle_and_sort(mapped_data):
    """Since we are computing global statistics, this simply collects all mapped data."""
    sum_x = 0
    sum_x2 = 0
    count = 0
    for x, x2, c in mapped_data:
        sum_x += x
        sum_x2 += x2
        count += c
    return sum_x, sum_x2, count

### 3.3 Reduce Step: Compute sum, sum of squares, and count to derive the mean and variance.

In [7]:
def reduce_phase(shuffled_data):
    """Computes mean and variance from aggregated data."""
    sum_x, sum_x2, count = shuffled_data
    if count == 0:
        return None, None  # Avoid division by zero

    mean = sum_x / count
    variance = (sum_x2 / count) - (mean ** 2)
    
    return mean, variance

### 3.4 Putting It All Together

In [8]:
# 1. Map Phase
mapped = map_phase("numbers.txt")  # Input file with numbers

# 2. Shuffle & Sort Phase
shuffled = shuffle_and_sort(mapped)

# 3. Reduce Phase
mean, variance = reduce_phase(shuffled)

# Print Results
print(f"Mean: {mean}")
print(f"Variance: {variance}")

Mean: 30.813900000000007
Variance: 149.0857957899999


## 4. MapReduce Use Case: Linear Regression
We want to fit a linear model: $$y = mx + b$$ using gradient descent, which iteratively updates parameters m (slope) and b (intercept) based on the gradient: $$m:= m - \alpha \frac{1}{N}\sum_i((mx_i+b-y_i)x_i)$$
where:
- $\alpha$ is the learning rate.
- $N$ is the number of data points.

### 4.1 Mapper Function
Each mapper reads a subset of the data and computes the partial gradients for m and b:

In [9]:
def map_phase(input_file, m, b):
    """Computes the partial gradients for m and b for each chunk."""
    with open(input_file, 'r') as f:
        for line in f:
            x, y = map(float, line.strip().split(','))  # Read X and Y
            error = (m * x + b) - y  # Compute error
            gradient_m = error * x   # Partial derivative w.r.t. m
            gradient_b = error       # Partial derivative w.r.t. b
            yield (1, (gradient_m, gradient_b, 1))  # Emit (key=1, values=(∂m, ∂b, count=1))

### 4.2 Shuffle & Sort Phase
This step groups all partial gradients from different mappers.

In [10]:
def shuffle_and_sort(mapped_data):
    """Aggregates partial gradients from all mappers."""
    sum_gradient_m = sum_gradient_b = count = 0
    for _, (gradient_m, gradient_b, c) in mapped_data:
        sum_gradient_m += gradient_m
        sum_gradient_b += gradient_b
        count += c
    return sum_gradient_m, sum_gradient_b, count

### 4.3 Reducer Function
The reducer aggregates the gradients and updates m and b:

In [11]:
def reduce_phase(shuffled_data, m, b, alpha):
    """Updates m and b using the averaged gradients."""
    sum_gradient_m, sum_gradient_b, count = shuffled_data
    if count == 0:
        return m, b  # Avoid division by zero

    # Compute the average gradients
    avg_gradient_m = sum_gradient_m / count
    avg_gradient_b = sum_gradient_b / count

    # Update parameters using gradient descent
    m -= alpha * avg_gradient_m
    b -= alpha * avg_gradient_b

    return m, b

### 4.4 Put It All Together

In [12]:
# Hyperparameters
alpha = 0.1  # Learning rate
iterations = 500  # Number of iterations

# Initialize parameters
m, b = 0.0, 0.0  # Initial guess

# Run gradient descent for multiple iterations
for i in range(iterations):
    print(f"Iteration {i+1}: m = {m:.4f}, b = {b:.4f}")

    # Step 1: Map (Compute Partial Gradients)
    mapped = map_phase("data.csv", m, b)

    # Step 2: Shuffle & Sort (Aggregate Gradients)
    shuffled = shuffle_and_sort(mapped)

    # Step 3: Reduce (Update Parameters)
    m, b = reduce_phase(shuffled, m, b, alpha)

# Final Model
print(f"Final Linear Regression Model: y = {m:.4f}x + {b:.4f}")

Iteration 1: m = 0.0000, b = 0.0000
Iteration 2: m = 2.1200, b = 0.5600
Iteration 3: m = 1.7400, b = 0.4280
Iteration 4: m = 1.8176, b = 0.4232
Iteration 5: m = 1.8113, b = 0.3956
Iteration 6: m = 1.8202, b = 0.3727
Iteration 7: m = 1.8262, b = 0.3493
Iteration 8: m = 1.8326, b = 0.3265
Iteration 9: m = 1.8388, b = 0.3041
Iteration 10: m = 1.8449, b = 0.2821
Iteration 11: m = 1.8509, b = 0.2604
Iteration 12: m = 1.8568, b = 0.2391
Iteration 13: m = 1.8626, b = 0.2181
Iteration 14: m = 1.8683, b = 0.1976
Iteration 15: m = 1.8739, b = 0.1773
Iteration 16: m = 1.8794, b = 0.1574
Iteration 17: m = 1.8848, b = 0.1378
Iteration 18: m = 1.8902, b = 0.1186
Iteration 19: m = 1.8954, b = 0.0997
Iteration 20: m = 1.9006, b = 0.0811
Iteration 21: m = 1.9056, b = 0.0628
Iteration 22: m = 1.9106, b = 0.0449
Iteration 23: m = 1.9155, b = 0.0272
Iteration 24: m = 1.9203, b = 0.0098
Iteration 25: m = 1.9250, b = -0.0072
Iteration 26: m = 1.9297, b = -0.0240
Iteration 27: m = 1.9342, b = -0.0405
Iterati

## 5. ✅ Advantages and Limitations
✅ Advantages
- Simplifies parallel programming.

- Scales horizontally across distributed systems.

- Built-in fault tolerance.

❌ Limitations
- Poor performance for iterative or real-time algorithms.

- Less flexible than general-purpose distributed computing frameworks​


## Homework Assignments
Please use the same `input.txt` file in section 2, modify mapper/ Reducer function to count all the words with length $>=4$.

In [17]:
def map_phase(input_file):
    """Reads lines from input_file, yields (word, 1) for each word."""
    with open(input_file, 'r') as f:
        for line in f:
            words = line.strip().split()
            for word in words:
                if len(word) >= 4:
                    yield (word.lower(), 1)   # Convert to lower case

In [19]:
def shuffle_and_sort(mapped_data):
    """Groups the mapped data by key (word)."""
    sorted_data = {}
    for key, value in mapped_data:
        if key not in sorted_data:
            sorted_data[key] = []
        sorted_data[key].append(value)
    return sorted_data

In [18]:
def reduce_phase(shuffled_data):
    """For each word key, sum up all the values."""
    reduced_data = {}
    for key, values in shuffled_data.items():
        reduced_data[key] = sum(values)
    return reduced_data

In [20]:
# 1. Map
mapped = map_phase("input.txt")

# 2. Shuffle/Sort
shuffled = shuffle_and_sort(mapped)

# 3. Reduce
reduced_result = reduce_phase(shuffled)

# Print or save results
for word, count in reduced_result.items():
    print(f"{word}: {count}")

mapreduce: 2
programming: 1
model: 1
processing: 1
large: 1
data: 2
sets.: 1
inspired: 1
reduce: 2
functions: 1
commonly: 1
used: 2
functional: 1
programming.: 1
function: 2
applies: 1
transformation: 1
each: 1
element: 1
collection,: 1
while: 1
aggregates: 1
results.: 1
enables: 1
distributed: 1
computing: 1
across: 1
many: 1
machines.: 1
this: 1
approach: 1
widely: 1
processing,: 1
including: 1
analysis: 1
indexing.: 1
